# Example

This example demonstrates the whole process from initial atomic structure to training, evaluation and prediction. 

It includes:
1. Read input atomic structures (saved as extxyz files) and create descriptors and their derivatives.

2. Read inputs and outputs into a Data object.

3. Create tensorflow dataset for training.

4. Train the potential, use it for prediction and save the trained model.

5. Load the trained model for retraining or prediction.


The code has been run with Tensorflow 2.6.

In [1]:
import atomdnn

# 'float64' is used for reading data and train by default
atomdnn.data_type = 'float64'

# force and stress are evaluated by default, 
# if one only need to compute potential energy, then set compute_force to false
atomdnn.compute_force = True

# default value is for converting ev/A^3 to GPa
# note that: the predicted positive stress means tension and negative stress means compression
stress_unit_convert = 160.2176 

import numpy as np
import tensorflow as tf
from atomdnn import data
from atomdnn.data import Data
from atomdnn.data import *
from atomdnn.io import *
from atomdnn import network
from atomdnn.network import Network

**1. Read input atomic structures (saved as extxyz files) and create descriptors and their derivatives**


In [2]:
descriptor = {'name': 'acsf', 
              'cutoff': 6.5,
              'etaG2':[0.01,0.025,0.05,0.075,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.8,1,1.5,2,3,5,10], 
              'etaG4': [0.01], 
              'zeta': [0.08,0.1,0.15,0.2,0.3,0.35,0.5,0.6,0.8,1.,1.5,2.,3.0,4.,5.5,7.0,10.0,25.0,50.0,100.0],
              'lambda': [1.0, -1.0]}

# define lammps excutable (serial or mpi) 
# LAMMPS has to be compiled with the added compute and dump_local subrutines (inside atomdnn/lammps)
lmpexe = 'lmp_serial' 

In [3]:
xyzfile_path = './extxyz' 
xyzfile_name = 'example_extxyz.*' # a serials of files like example_extxyz.1, example_extxyz.2, ...example_extxyz.n
descriptors_path = './descriptors'
descriptor_filename = 'dump_fp' # a serials of dump_fp.* files will be created
der_filename ='dump_der'

# this will create a serials of files for descriptors and their derivatives inside descriptors_path
# by default, descriptor files are saved as 'dump_fp.*' and derivatives are saved as 'dump_der.*'
create_descriptors(xyzfile_path = xyzfile_path, \
                   xyzfile_name = xyzfile_name, \
                   lmpexe = lmpexe, \
                   descriptors_path = descriptors_path, \
                   descriptor = descriptor, \
                   descriptor_filename = descriptor_filename, \
                   der_filename = der_filename)

Start creating fingerprints ...
  so far finished for 5 images ...


/bin/bash: lmp_serial: command not found
/bin/bash: lmp_serial: command not found
/bin/bash: lmp_serial: command not found
/bin/bash: lmp_serial: command not found
/bin/bash: lmp_serial: command not found
/bin/bash: lmp_serial: command not found


  so far finished for 10 images ...
Finish creating descriptors and their derivatives from total 10 images.
It took 0.42 seconds.


/bin/bash: lmp_serial: command not found
/bin/bash: lmp_serial: command not found
/bin/bash: lmp_serial: command not found
/bin/bash: lmp_serial: command not found


**2. Read inputs and outputs into a Data object** 

In [3]:
# create a Data object
grdata = Data()

# read inputs: descriptors and their derivatives
fp_filename = descriptors_path + '/dump_fp.*'
der_filename = descriptors_path + '/dump_der.*'

grdata.read_inputdata(fp_filename = fp_filename,der_filename = der_filename)


Reading fingerprints data from LAMMPS dump files ./descriptors/dump_fp.i
  Finish reading fingerprints from total 10 images.

  image number = 10
  max number of atom = 4
  number of fingerprints = 59
  type of atoms = 1

Reading derivative data from a series of files ./descriptors/dump_der.i
This may take a while ...
  Finish reading dGdr derivatives from total 10 images.

  Pad zeros to derivatives data if needed ...
  Pading finished: 9 images derivatives have been padded with zeros.

  image number = 10
  max number of blocks = 150
  number of fingerprints = 59

  It took 0.34 seconds to read the derivatives data.


In [4]:
# read outputs: potential energy, force and stress from extxyz files
grdata.read_outputdata(xyzfile_path=xyzfile_path, xyzfile_name=xyzfile_name)

Reading outputs from extxyz files ...
  Finish reading outputs from total 10 images.



**3. Create tensorflow dataset for training**

In [5]:
# convert data to tensors
grdata.convert_data_to_tensor()

Conversion may take a while for large datasets...
It took 0.0923 second.


In [6]:
# create tensorflow dataset
tf_dataset = tf.data.Dataset.from_tensor_slices((grdata.input_dict,grdata.output_dict))

# save the dataset
dataset_name = './temp_tfdataset'
tf.data.experimental.save(tf_dataset, dataset_name)

**Note: The above three steps just need to be done once for one data set, the training only uses the saved tensorflow dataset.**

**4. Load the dataset and train the model**

In [13]:
# load tensorflow dataset, for Tensorflow version lower than 2.6, need to specify element_spec.
# The process of reading data and creating dataset is discussed in 'Data pipeline' section.

dataset = tf.data.experimental.load('example_tfdataset')

TypeError: load() missing 1 required positional argument: 'element_spec'

In [14]:
# split the data to training, validation and testing sets

train_dataset, val_dataset, test_dataset = split_dataset(dataset,0.7,0.2,0.1,shuffle=True)

NameError: name 'dataset' is not defined

In [ ]:
# Build the network
# See section 'Training' for detailed description on Network object.

elements = ['C']
act_fun = 'relu' # activation function
nfp = get_fingerprints_num(dataset) # number of fingerprints (or descriptors)
arch = [30,30] # NN layers

model = Network(elements = elements,\
                num_fingerprints = nfp,\
                arch = arch,\
                activation_function = act_fun)

In [ ]:
# Train the model 

opt = 'Adam' # optimizer
loss_fun = 'rmse' # loss function
scaling = 'std' # scaling the traning data with standardization
lr = 0.02 # learning rate
loss_weights = {'pe' : 1, 'force' : 1, 'stress': 0.1} # the weights in loss function

model.train(train_dataset, val_dataset, \
            optimizer=opt, \
            loss_fun = loss_fun, \
            batch_size=30, \
            lr=lr, \
            epochs=30, \
            scaling=scaling, \
            loss_weights=loss_weights, \
            compute_all_loss=True, \
            shuffle=True, \
            append_loss=True)

In [ ]:
# plot the training loss

model.plot_loss(start_epoch=1)

In [ ]:
# Evaluate using the first 5 data in test dataset

model.evaluate(test_dataset.take(5),return_prediction=False)

In [ ]:
# prediction using the first 5 data in test dataset

input_dict = get_input_dict(test_dataset.take(5))
model.predict(input_dict)

In [ ]:
# save the trained model

# we re-write the descriptor here to empasize that it should be the same one defined above
descriptor = {'name': 'acsf', 
              'cutoff': 6.5001,
              'etaG2':[0.01,0.025,0.05,0.075,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.8,1,1.5,2,3,5,10], 
              'etaG4': [0.01], 
              'zeta': [0.08,0.1,0.15,0.2,0.3,0.35,0.5,0.6,0.8,1.,1.5,2.,3.0,4.,5.5,7.0,10.0,25.0,50.0,100.0],
              'lambda': [1.0, -1.0]}

save_dir = 'example.tfdnn'
network.save(model,save_dir,descriptor=descriptor)

**5. Load the trained model for continuous training and prediction**

In [ ]:
imported_model = network.load(save_dir)

# Re-train the model 
loss_weights = {'pe' : 1, 'force' : 1, 'stress': 0.1}

opt = 'Adam'
loss_fun = 'rmse'
scaling = 'std'

model.train(train_dataset, val_dataset, \
            optimizer=opt, \
            loss_fun = loss_fun, \
            batch_size=30, \
            lr=0.02, \
            epochs=5, \
            scaling=scaling, \
            loss_weights=loss_weights, \
            compute_all_loss=True, \
            shuffle=True, \
            append_loss=True)

In [ ]:
imported_model.evaluate(test_dataset.take(5),return_prediction=False)

In [ ]:
input_dict = get_input_dict(test_dataset.take(5))
imported_model.predict(input_dict)